In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import feature_manager as fma
import rl.env_simple_crypto_trade as env
import importlib
import rl.models as rla
import config as cf
import numpy as np
import os
import time

In [ ]:
importlib.reload(fma)

fm = fma.FeatureManager(target_col="trade_signal")
fm.import_trading_data(symbol="BTCUSDT",trade_timeframe="4h")
fm.import_macro_data(symbol="BTCUSDT",macro_timeframe="1d")
fm.import_super_data(symbol="BTCUSDT",super_timeframe="1w")

fm.build_features(
    lags = 1,
    macro_lags = 1,
    super_lags = 1,
    features = cf.BITCOIN_EXTERNAL_INDICATORS + cf.TRADING_TA_INDICATORS + cf.CANDLESTICK_INDICATORS,
    macro_features = cf.MACRO_TA_INDICATORS + cf.CANDLESTICK_INDICATORS,
    super_features = cf.SUPER_TA_INDICATORS + cf.CANDLESTICK_INDICATORS,
)

In [ ]:
train = fm.df.iloc[-4000:-2000]
trade = fm.df.iloc[-2000:]

In [ ]:
importlib.reload(env)

env_kwargs = cf.TRADE_ENV_PARAMETER
state_space = 3 + len(fm.cols)

train_env = env.CryptoTradingEnv(
    df = train, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

trade_env = env.CryptoTradingEnv(
    df = trade, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)
full_env = env.CryptoTradingEnv(
    df = fm.df, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

In [ ]:
importlib.reload(rla)
agent = rla.DRLTradeAgent(env=full_env)

In [ ]:
df = pd.read_csv("../out/gamma_99_profit_315000_steps.csv")
profit_list = df["mean"].to_list()
std_list = df["std"].to_list()
last_timestep = df["timestep"].iloc[-1]

In [ ]:
%store profit_list
%store std_list

In [32]:
test_env = full_env
test_laps = 5

# profit_list = []
# std_list = []

start_timestep = 320_000
end_timestep = 1_000_000 + 1
frequency = 5_000

for step in range(start_timestep,end_timestep,frequency):
    print(f"Timestep {step}:")

    # path = cf.CHECKPOINT_CALLBACK["save_dir"] + f"full_add_cci_bb/rl_model_{step}_steps.zip"
    checkpoint_path = f"full_add_cci_bb/rl_model_{step}_steps"

    # while(not os.path.exists(path)):
    #     time.sleep(10)
    #     continue    
    
    # print(path)

    model = agent.load_model_from_checkpoint("ppo",checkpoint_path)

    profits = []
    costs = []
    for i in range(0,test_laps):
        agent.predict(model,test_env,render=False)
        result = agent.make_result_data()
        profit = result.iloc[-1,:]["cumsum_trade_profit"]
        profits.append(profit)
        print(f"\tLap {i}/{test_laps}: Profit {profit}")

    mean_profit = np.array(profits).mean()
    std_profit = np.array(profits).std()
    profit_list.append(mean_profit)
    std_list.append(std_profit)
    print(f"\t Average profit: {mean_profit}, sdt: {std_profit}")
    %store profit_list 
    %store std_list

Timestep 320000:
Successfully load model from ../saved_models/checkpoint/full_add_cci_bb/rl_model_320000_steps
	Lap 0/5: Profit 2.22018116769724
	Lap 1/5: Profit 1.865512981579633
	Lap 2/5: Profit 0.44168619428649925
	Lap 3/5: Profit 1.3332649631266116
	Lap 4/5: Profit 0.8132263781977909
	 Average profit: 1.3347743369775549, sdt: 0.6534905885919471
Stored 'profit_list' (list)
Stored 'std_list' (list)
Timestep 325000:
Successfully load model from ../saved_models/checkpoint/full_add_cci_bb/rl_model_325000_steps
	Lap 0/5: Profit 1.989191130548898
	Lap 1/5: Profit 2.008695218952635
	Lap 2/5: Profit 0.6619644585534359
	Lap 3/5: Profit 1.183603899121248
	Lap 4/5: Profit 1.1875731319999419
	 Average profit: 1.4062055678352319, sdt: 0.5204060760011667
Stored 'profit_list' (list)
Stored 'std_list' (list)
Timestep 330000:
Successfully load model from ../saved_models/checkpoint/full_add_cci_bb/rl_model_330000_steps
	Lap 0/5: Profit 1.4864900786437145
	Lap 1/5: Profit 2.0044402603812728
	Lap 2/5: 

KeyboardInterrupt: 

In [ ]:
%store